<a href="https://colab.research.google.com/github/mzohaibnasir/Llamaindex/blob/main/llamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# llamaIndex

In [ ]:
from google.colab import userdata
openai_key = userdata.get('OPENAI_API_KEY')
openai_key

In [ ]:
! pip install llama-index openai pypdf python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()



False

In [ ]:
os.environ['OPENAI_API_KEY'] = openai_key
# os.environ['OPENAI_API_KEY']

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
reader = SimpleDirectoryReader('./PDFs/')
data = reader.load_data()
len(data)

63

In [ ]:
print(data[0])

Doc ID: 64db49e9-4d5f-4a74-9466-2ea9dedee053
Text: Health Insurance Coverage Status and Type  by Geography: 2021
and 2022 American Community Survey Briefs ACSBR-015Issued September
2023Douglas Conway and Breauna Branch INTRODUCTION Demographic shifts
as well as economic and govern- ment policy changes can affect
people’s access to health coverage. For example, between 2021 and
2022, the labor ma...


In [ ]:
print(dict(data[0]).keys())

dict_keys(['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'text', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator'])


In [ ]:
import json
json.dumps(dict(data[0]))

In [ ]:
# convert into index
index = VectorStoreIndex.from_documents(data,show_progress=True)
index

Parsing nodes:   0%|          | 0/63 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/117 [00:00<?, ?it/s]

In [ ]:
# query engine
query_engine = index.as_query_engine()
query_engine

In [ ]:
response = query_engine.query("What Are these pdfs about")
print(response)

The PDFs contain statistical data related to different states and territories in the United States, including various numerical values and percentages associated with each location.


In [45]:
response = query_engine.query("What is 'Attention is All you need'?")
print(response)

'Attention is All you need' is a paper that introduced the Transformer model, a type of neural network architecture that has been widely used in natural language processing tasks.


There are also some terms you'll encounter that refer to steps within each of these stages.

## Loading stage#
### Nodes and Documents:
A Document is a container around any data source - for instance, a PDF, an API output, or retrieve data from a database. A Node is the atomic unit of data in LlamaIndex and represents a "chunk" of a source Document. Nodes have metadata that relate them to the document they are in and to other nodes.

### Connectors:
A data connector (often called a Reader) ingests data from different data sources and data formats into Documents and Nodes.

## Indexing Stage#
### Indexes:
Once you've ingested your data, LlamaIndex will help you index the data into a structure that's easy to retrieve. This usually involves generating vector embeddings which are stored in a specialized database called a vector store. Indexes can also store a variety of metadata about your data.

### Embeddings:
 LLMs generate numerical representations of data called embeddings. When filtering your data for relevance, LlamaIndex will convert queries into embeddings, and your vector store will find data that is numerically similar to the embedding of your query.

## Querying Stage#
### Retrievers:
A retriever defines how to efficiently retrieve relevant context from an index when given a query. Your retrieval strategy is key to the relevancy of the data retrieved and the efficiency with which it's done.

### Routers:
A router determines which retriever will be used to retrieve relevant context from the knowledge base. More specifically, the RouterRetriever class, is responsible for selecting one or multiple candidate retrievers to execute a query. They use a selector to choose the best option based on each candidate's metadata and the query.

### Node Postprocessors:
 A node postprocessor takes in a set of retrieved nodes and applies transformations, filtering, or re-ranking logic to them.

### Response Synthesizers:
 A response synthesizer generates a response from an LLM, using a user query and a given set of retrieved text chunks.

## Putting it all together#
There are endless use cases for data-backed LLM applications but they can be roughly grouped into three categories:

1. Query Engines:
 A query engine is an end-to-end pipeline that allows you to ask questions over your data. It takes in a natural language query, and returns a response, along with reference context retrieved and passed to the LLM.

2. Chat Engines: A chat engine is an end-to-end pipeline for having a conversation with your data (multiple back-and-forth instead of a single question-and-answer).

3. Agents: An agent is an automated decision-maker powered by an LLM that interacts with the world via a set of tools. Agents can take an arbitrary number of steps to complete a given task, dynamically deciding on the best course of action rather than following pre-determined steps. This gives it additional flexibility to tackle more complex tasks.



In [46]:
# from llama_index.responses import PPrintResponse

from llama_index.core.data_structs import Node
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer()


synthesized_query_engine = index.as_query_engine(response_synthesizer=response_synthesizer)
synthesized_query_engine



response = synthesized_query_engine.query("What is 'Attention is All you need'?")
print(response)

'Attention is All you need' is a paper that introduced the Transformer model, a type of neural network architecture that has been widely used in natural language processing tasks.


1. **VectorIndexRetriever**: A retriever that uses a vector index to retrieve documents based on semantic similarity.
2. **RetrieverQueryEngine**: A query engine that uses a retriever to retrieve documents and then answers questions based on the retrieved documents.


In [76]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.response.pprint_utils import pprint_response

from pprint import pprint

vRetriever = VectorIndexRetriever(index= index, similarity_top_k=4)
retrieverQueryEngine = RetrieverQueryEngine(retriever = vRetriever)
# query("What are thesse pdfs about?")
response = retrieverQueryEngine.query("What are thesse pdfs about?")
pprint_response(response,show_source=True)


Final Response: The PDFs contain data related to various states in the
United States, including estimates and margins of error for different
categories such as income, employment, and industry sectors. The
information is sourced from the U.S. Census Bureau's 2018 Survey of
Income and Program Participation.
______________________________________________________________________
Source Node 1/4
Node ID: 2eb55b42-ac95-4444-8fe6-c5ad057ccd88
Similarity: 0.7604724557932931
Text: 9 0.2 3.8 0.2 4.3 0.2 *7.9 0.2 ^31.7 0.4 *39.0 0.4 Minnesota  .
4.5 0.3 5.2 0.3 2.8 0.2 3.3 0.2 6.4 0.3 29.2 0.5 48.7 0.6 Mississippi
. 8.8 0.5 10.3 0.5 5.7 0.5 *5.1 0.4 ^10.1 0.6 31.6 0.8 28.4 0.7
Missouri  . ^6.3 0.3 6.8 0.3 4.1 0.2 4.2 0.3 8.5 0.3 ^33.2 0.5 *36.8
0.4 Montana  . 5.8 0.6 6.3 0.6 3.6 0.5 4.4 0.6 9.3 0.8 ^33.9 1.3 36.7
1.4 Nebra...
______________________________________________________________________
Source Node 2/4
Node ID: 87e2235b-d13f-41a6-bb17-e3fcf745e1ee
Similarity: 0.7593208768087353
Text: 1 

In [79]:
# for similarity threshold
vRetriever = VectorIndexRetriever(index= index, similarity_top_k=4)

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.76)
RetrieverQueryEngine = RetrieverQueryEngine(retriever = vRetriever,node_postprocessors=[postprocessor])
# query("What are thesse pdfs about?")
response = RetrieverQueryEngine.query("What are thesse pdfs about?")
pprint_response(response,show_source=True)


Final Response: The PDF seems to contain statistical data related to
different states in the United States, possibly focusing on various
metrics or indicators for each state.
______________________________________________________________________
Source Node 1/1
Node ID: 2eb55b42-ac95-4444-8fe6-c5ad057ccd88
Similarity: 0.7604724557932931
Text: 9 0.2 3.8 0.2 4.3 0.2 *7.9 0.2 ^31.7 0.4 *39.0 0.4 Minnesota  .
4.5 0.3 5.2 0.3 2.8 0.2 3.3 0.2 6.4 0.3 29.2 0.5 48.7 0.6 Mississippi
. 8.8 0.5 10.3 0.5 5.7 0.5 *5.1 0.4 ^10.1 0.6 31.6 0.8 28.4 0.7
Missouri  . ^6.3 0.3 6.8 0.3 4.1 0.2 4.2 0.3 8.5 0.3 ^33.2 0.5 *36.8
0.4 Montana  . 5.8 0.6 6.3 0.6 3.6 0.5 4.4 0.6 9.3 0.8 ^33.9 1.3 36.7
1.4 Nebra...


In [70]:
# Node post process
from llama_index.core.postprocessor import SimilarityPostprocessor
# from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.data_structs import Node
from llama_index.core.schema import NodeWithScore

nodes = [
    NodeWithScore(node=Node(text="ABC"), score=0.7),
    NodeWithScore(node=Node(text="XYZ"), score=0.8),
]

# similarity postprocessor: filter nodes below 0.75 similarity score
processor = SimilarityPostprocessor(similarity_cutoff=0.75)
filtered_nodes = processor.postprocess_nodes(nodes)
pprint(filtered_nodes)

[NodeWithScore(node=TextNode(id_='aefe0dc6-6514-4b93-99a4-0073f7582405', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='XYZ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8)]
[NodeWithScore(node=TextNode(id_='aefe0dc6-6514-4b93-99a4-0073f7582405', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='XYZ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8)]
